In [23]:
import gzip
import json
import os
import shutil
from collections import Counter
from pathlib import Path

from src.utils import extract_frames_to_dir
from tqdm import tqdm

In [17]:
with open("experiment_metadata.json") as file:
    experiment_metadata = json.load(file)
    recordings_metadata = experiment_metadata["trial_recordings_metadata"]

# Check if metadata is correct

In [18]:
# Count unique recording IDs
unique_ids_count = len(recordings_metadata)
assert unique_ids_count == 14, f"Expected 14 unique IDs, but found {unique_ids_count}."
print(f"Unique ID count: {unique_ids_count}")

# Collect all object names from the JSON data
all_objects = []
for record in recordings_metadata.values():
    all_objects.extend(record.get("objects", []))

# Count the frequency of each object name
object_counter = Counter(all_objects)

# Assert that there are exactly 15 different object names
assert len(object_counter) == 15, (
    f"Expected 15 unique objects, but found {len(object_counter)}."
)

# Print the unique object count and the frequency of each object
print(f"Unique object count: {len(object_counter)}")
print("Object frequencies:")
for obj, freq in object_counter.items():
    print(f"{obj}: {freq}")

Unique ID count: 14


TypeError: unhashable type: 'list'

# Check if the expected number of recordings are present

In [19]:
# Define the base paths using absolute paths
RAW_EXPERIMENT_RECORDINGS_PATH = os.path.abspath("data/raw/experiment_recordings/")
RAW_LABELING_RECORDINGS_PATH = os.path.abspath("data/raw/labeling_recordings/")

# List all subdirectories (full paths) in the experiment recordings folder
experiment_dirs = [
    os.path.join(RAW_EXPERIMENT_RECORDINGS_PATH, d)
    for d in os.listdir(RAW_EXPERIMENT_RECORDINGS_PATH)
    if os.path.isdir(os.path.join(RAW_EXPERIMENT_RECORDINGS_PATH, d))
]

# List all subdirectories (full paths) in the labeling recordings folder
labeling_dirs = [
    os.path.join(RAW_LABELING_RECORDINGS_PATH, d)
    for d in os.listdir(RAW_LABELING_RECORDINGS_PATH)
    if os.path.isdir(os.path.join(RAW_LABELING_RECORDINGS_PATH, d))
]

# Assert the counts
assert len(experiment_dirs) == 14, (
    f"Expected 14 recordings in experiment recordings folder, but found {len(experiment_dirs)}."
)
assert len(labeling_dirs) == 2, (
    f"Expected 2 recordings in labeling recordings folder, but found {len(labeling_dirs)}."
)

print("Experiment recordings count:", len(experiment_dirs))
print("Labeling recordings count:", len(labeling_dirs))

Experiment recordings count: 14
Labeling recordings count: 2


# Preprocess the necessary recording files and save them to processed directory 

In [20]:
PROCESSED_RECORDINGS_PATH = Path("data/recordings/")
if PROCESSED_RECORDINGS_PATH.exists():
    shutil.rmtree(PROCESSED_RECORDINGS_PATH)
PROCESSED_RECORDINGS_PATH.mkdir(parents=True, exist_ok=True)

In [21]:
def preprocess_recording(recording_path, output_path, assert_in_metadata=True):
    # read metadata
    with open(recording_path / "recording.g3") as file:
        metadata = json.load(file)
        uuid = metadata["uuid"]

        if assert_in_metadata:
            # Check if the UUID is in the metadata
            assert uuid in recordings_metadata.keys(), (
                f"UUID {uuid} not found in metadata."
            )

    # copy the recording to the output path with name uuid.mp4 (from recording_path/scenevideo.mp4)
    scene_video_path = recording_path / "scenevideo.mp4"
    output_video_path = output_path / f"{uuid}.mp4"
    shutil.copy(scene_video_path, output_video_path)

    # unzip gaze data and save as uuid.tsv (from recording_path/gazedata.gz)
    with gzip.open(recording_path / "gazedata.gz") as zipFile:
        with open(output_path / f"{uuid}.tsv", "wb") as unzippedFile:
            for line in zipFile:
                unzippedFile.write(line)


for recording in experiment_dirs:
    preprocess_recording(Path(recording), PROCESSED_RECORDINGS_PATH)

for recording in labeling_dirs:
    preprocess_recording(
        Path(recording), PROCESSED_RECORDINGS_PATH, assert_in_metadata=False
    )

# Extract Frames for all Recordings

In [25]:
recordings = list(PROCESSED_RECORDINGS_PATH.glob("*.mp4"))
recording_uuids = [recording.stem for recording in recordings]

RECORDING_FRAMES_PATH = Path("data/recording_frames/")
if RECORDING_FRAMES_PATH.exists():
    shutil.rmtree(RECORDING_FRAMES_PATH)
RECORDING_FRAMES_PATH.mkdir(parents=True, exist_ok=True)

for recording_uuid in tqdm(recording_uuids, desc="Extracting frames"):
    recording_video_path = PROCESSED_RECORDINGS_PATH / f"{recording_uuid}.mp4"
    recording_frames_path = RECORDING_FRAMES_PATH / recording_uuid
    recording_frames_path.mkdir(parents=True, exist_ok=True)
    extract_frames_to_dir(recording_video_path, recording_frames_path)

Extracting frames: 100%|██████████| 16/16 [02:29<00:00,  9.37s/it]


# Build the sqlite database for the application 

In [6]:
from datetime import datetime, timedelta

from sqlalchemy.orm import Session
from src.db import Base, engine
from src.db.models import CalibrationRecording, Recording, SimRoom, SimRoomClass

In [7]:
# Prepare an empty database
Base.metadata.drop_all(bind=engine)
Base.metadata.create_all(bind=engine)

with Session(engine) as session:
    # Create a SimRoom
    sim_room = SimRoom(name="Controlled Experiment Room")
    session.add(sim_room)
    # Flush to get the ID without committing the transaction
    session.flush()

    # Create Classes:
    for i, object_name in enumerate(object_counter.keys()):
        sim_room_class = SimRoomClass(
            id=i + 1,
            sim_room_id=sim_room.id,
            class_name=object_name,
        )
        session.add(sim_room_class)

    # Create recordings
    for recording_path in experiment_dirs + labeling_dirs:
        with open(os.path.join(recording_path, "recording.g3")) as file:
            metadata = json.load(file)
            uuid = metadata["uuid"]
            visible_name = metadata["name"]
            created = datetime.strptime(metadata["created"], "%Y-%m-%dT%H:%M:%S.%fZ")
            created = created.isoformat()
            duration = str(timedelta(seconds=metadata["duration"]))
            folder_name = "not important"
            scene_video_url = "not important"
            gaze_data_url = "not important"

        with open(os.path.join(recording_path, "meta/participant")) as file:
            meta = json.load(file)
            participant = meta["name"]

            # Manually override wrong participant name
            if participant == "":
                participant = "Ilian Opname 12"
            elif participant == "Ilian Opname 12":
                participant = "Ilian Opname 14"
            elif participant == "Ilian Opname 10 b":
                participant = "Ilian Opname 10"

        recording = Recording(
            uuid=uuid,
            visible_name=visible_name,
            created=created,
            duration=duration,
            folder_name=folder_name,
            scene_video_url=scene_video_url,
            gaze_data_url=gaze_data_url,
            participant=participant,
        )
        session.add(recording)
        # Flush to get the ID without committing the transaction
        session.flush()

        calibration_recording = CalibrationRecording(
            sim_room_id=sim_room.id, recording_uuid=uuid
        )
        session.add(calibration_recording)

    session.commit()